In [34]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, FloatSlider, IntSlider, VBox, HBox, Output, Layout, HTML, ToggleButton, Label

# --- Data Generation Helper ---
def generate_data():
    global slope, intercept, noise_std, X, y, X_b
    slope = np.random.uniform(-5, 5)
    intercept = np.random.uniform(-10, 10)
    noise_std = np.random.uniform(0.5, 2.0)
    X = 6 * np.random.rand(100, 1) - 3
    y = intercept + slope * X + np.random.normal(0, noise_std, size=(100, 1))
    X_b = np.c_[np.ones((100, 1)), X]

generate_data()

# Always show all four quadrants:
x_min, x_max = -3, 3
y_min, y_max = -20, 20

# Initialize weights
theta = np.zeros((2, 1))
output = Output()
step_info = HTML()
step_counter = 0

# --- Widgets ---
title_html = HTML(value="<h2>Regression Gradient Descent Visualizer</h2>")

step_button = Button(
    description='Step',
    layout=Layout(width='150px'),
    button_style='primary'   # makes it blue
)
reset_button = Button(description='Reset Line', layout=Layout(width='150px'))
refresh_button = Button(description='Refresh Data', layout=Layout(width='150px'))

# Sliders with labels
lr_label = Label(value='Learning rate:', layout=Layout(width='120px'))
lr_slider = FloatSlider(min=0.001, max=0.5, step=0.001, value=0.1, layout=Layout(width='200px'))
lr_row = HBox([lr_label, lr_slider])

batch_label = Label(value='Batch size:', layout=Layout(width='120px'))
batch_slider = IntSlider(min=1, max=100, step=5, value=100, layout=Layout(width='200px'))
batch_row = HBox([batch_label, batch_slider])

# Toggle styled as ON/OFF with label
random_init_label = Label(value='Random Init:', layout=Layout(width='120px'))
random_init_toggle = ToggleButton(
    value=False,
    description='OFF',
    tooltip='Toggle random initialization',
    layout=Layout(width='60px')
)
random_init_row = HBox([random_init_label, random_init_toggle])

# --- Plot Helper ---
def plot_current_line():
    with output:
        output.clear_output(wait=True)
        plt.figure(figsize=(8, 6))
        plt.scatter(X, y, color="blue", label="Data")
        x_plot = np.linspace(x_min, x_max, 100)
        y_plot = theta[0][0] + theta[1][0] * x_plot
        plt.plot(x_plot, y_plot, color="red", label="Line")
        plt.axhline(0, color='black', linewidth=1)
        plt.axvline(0, color='black', linewidth=1)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)
        plt.title("Regression Gradient Descent Visualizer")
        plt.legend()
        plt.show()

# --- Reset Logic ---
def reset_model():
    global theta, step_counter
    if random_init_toggle.value:
        theta = np.array([
            [np.random.uniform(-10, 10)],
            [np.random.uniform(-5, 5)]
        ])
    else:
        theta = np.zeros((2, 1))
    step_counter = 0
    plot_current_line()
    step_info.value = (
        f"<b>Step Number:</b> {step_counter} <br>"
        f"<b>Intercept:</b> {theta[0][0]:.4f} &nbsp;&nbsp; "
        f"<b>Slope:</b> {theta[1][0]:.4f}"
    )

# --- Refresh Data Logic ---
def refresh_data(b):
    generate_data()
    reset_model()

# --- Step Handler ---
def on_step_clicked(b):
    global theta, step_counter
    m = len(X_b)
    indices = np.random.choice(m, batch_slider.value, replace=False)
    X_batch = X_b[indices]
    y_batch = y[indices]
    old_theta = theta.copy()
    gradients = 2 / batch_slider.value * X_batch.T.dot(X_batch.dot(theta) - y_batch)
    theta -= lr_slider.value * gradients
    step_counter += 1
    plot_current_line()
    step_info.value = (
        f"<b>Step Number:</b> {step_counter} <br>"
        f"<b>Old Intercept:</b> {old_theta[0][0]:.4f} &nbsp;&nbsp; "
        f"<b>Old Slope:</b> {old_theta[1][0]:.4f} <br>"
        f"<b>Gradient Intercept:</b> {gradients[0][0]:.4f} &nbsp;&nbsp; "
        f"<b>Gradient Slope:</b> {gradients[1][0]:.4f} <br>"
        f"<b>New Intercept:</b> {theta[0][0]:.4f} &nbsp;&nbsp; "
        f"<b>New Slope:</b> {theta[1][0]:.4f}"
    )

# --- Toggle Handler ---
def on_toggle_change(change):
    random_init_toggle.description = 'ON' if change['new'] else 'OFF'
    reset_model()

# --- Event Wiring ---
step_button.on_click(on_step_clicked)
reset_button.on_click(lambda b: reset_model())
refresh_button.on_click(refresh_data)
random_init_toggle.observe(on_toggle_change, names='value')

# --- Layout ---
step_row = HBox([step_button])
reset_refresh_row = HBox([reset_button, refresh_button], layout=Layout(gap='10px'))

controls_column = VBox(
    [step_row, reset_refresh_row, lr_row, batch_row, random_init_row],
    layout=Layout(align_items='flex-start', gap='10px')
)

top_row = HBox([controls_column, step_info], layout=Layout(align_items='flex-start', gap='50px'))

ui = VBox([title_html, top_row, output],
          layout=Layout(padding='10px', border='1px solid lightgray'))

# --- Initial Plot ---
reset_model()

# --- Display ---
ui
